In [1]:
import tensorflow as tf

# https://github.com/patrick-kidger/tools
import tools

from base import fenics as fc
from base import ml

In [2]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [ ]:
### DNN hyperparameters
hidden_units = [100] * 7
# i.e. the number of fine grid points:
logits = (ml.fine_grid_fineness.t + 1) * (ml.fine_grid_fineness.x + 1)
processor = ml.ScaleCenter()
activation = tf.nn.crelu

### Training hyperparameters #1
drop_rate = 0.0
drop_type = 'dropout'
gradient_clip = 1.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir_base = './saved_models/grid_general/'
uuid = '2'
model_dir = ml.model_dir_str(model_dir_base, hidden_units, logits,
                             processor=processor, activation=activation, 
                             uuid=uuid)

dnn_factory = ml.DNNFactory(hidden_units, logits, drop_rate=drop_rate,
                            drop_type=drop_type, model_dir=model_dir, 
                            log_steps=log_steps, activation=activation, 
                            gradient_clip=gradient_clip, batch_norm=False,
                            processor=processor)

max_steps = 0
if TRAIN:
    gen_one_data = fc.GenGeneralSolutionOnGrid()

    ### Training hyperparameters #2
    batch_size = 50
    max_steps += 5000
    train_input_fn = ml.BatchData.from_func(gen_one_data=gen_one_data, 
                                            batch_size=batch_size)
    dnn = dnn_factory()
    dnn.train(input_fn=train_input_fn, max_steps=max_steps)

    batch_size = 500
    max_steps += 5000
    train_input_fn = ml.BatchData.from_func(gen_one_data=gen_one_data,
                                            batch_size=batch_size)
    dnn = dnn_factory()
    dnn.train(input_fn=train_input_fn, max_steps=max_steps)

    batch_size = 1500
    max_steps += 3000
    train_input_fn = ml.BatchData.from_func(gen_one_data=gen_one_data,
                                            batch_size=batch_size)
    dnn = dnn_factory()
    dnn.train(input_fn=train_input_fn, max_steps=max_steps)

INFO:tensorflow:FEniCS: Generated initial condition 0.7312339024777523 * (4 + sin(-0.9121446606198805 * x[0] + 1.5827205062945104) + sin(-0.10550845376068874 * x[0] + 1.1401032628776209) + sin(-1.3379709458847717 * x[0] + -1.4386269424580678) + sin(-0.6384514429279442 * x[0] + 0.7088183139786959)) / cosh(x[0] - 6.596549897761562) + 0.8168942112293548 * (2 + sin(0.22913899848475028 * x[0] + -0.21223534509279718) + sin(1.3726679230020873 * x[0] + 0.5052426079421721)) / cosh(x[0] - 13.050564951634911)
INFO:tensorflow:FEniCS: Making solution periodic by adding 0.00044850987982891603 * x[0] + -0.00448509879828916 to the initial condition.
INFO:tensorflow:Using config: {'_model_dir': './saved_models/grid_general/100x7_121__ScaleCenter_crelu_2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_di

Process ForkPoolWorker-7:
Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Process ForkPoolWorker-8:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/mlpatrick000/miniconda/envs/mltt/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/mlpatrick000/miniconda/envs/mltt/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/mlpatrick000/miniconda/envs/mltt/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/mlpatrick000/miniconda/envs/mltt/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Tracebac